In [22]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [23]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("Resources/lending_data.csv")
df_loan = pd.read_csv(file_path)

# Review the DataFrame
# YOUR CODE HERE!
df_loan.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [24]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_loan.loc[:,["loan_status"]]

# Separate the X variable, the features
X = df_loan.drop(columns="loan_status")

In [25]:
X = pd.get_dummies(X)

In [26]:
# Review the y variable Series
y

,loan_status
0,0
1,0
2,0
3,0
4,0
...,...
77531,1
77532,1
77533,1
77534,1


In [27]:
# Review the X variable DataFrame
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [28]:
# Check the balance of our target values
y["loan_status"].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [29]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [30]:
y_train

,loan_status
44997,0
13468,0
52163,0
5330,0
9982,0
...,...
28028,0
18051,0
23757,0
6607,0


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [31]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train["loan_status"])

predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test["loan_status"]}).tail(10)

,Prediction,Actual
76038,1,1
19119,0,0
45193,0,0
25211,0,0
61697,0,0
38069,0,0
36892,0,0
5035,0,0
40821,0,0
35030,0,0


In [32]:
# Score the testing data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9914878250103177
Testing Data Score: 0.9924164259182832


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [33]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test["loan_status"]}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [34]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score
# Display the accuracy score for the test dataset.
accuracy_score(y_test["loan_status"], predictions)

0.9924164259182832

In [35]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test["loan_status"], predictions)

array([[18679,    80],
       [   67,   558]], dtype=int64)

In [36]:
# Print the classification report for the model
from sklearn.metrics import classification_report
#target_names = ["Non-Healthy Loan", "Healthy Loan"]
print(classification_report(y_test["loan_status"], predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Precision and recall values would be more relevant because the cost of misclassification can be high. A high precision score is important to minimize false positives, which can lead to a loss of potential customers.

On the other hand, a high recall score is important to minimize false negatives, which can lead to significant financial losses.

This logistic regression model has high accuracy (0.99). When predicting healthy loans precision value is the maximum(1).The system also has a precision score (0.84) stating that there is a decent predicting score for the True positive , representing a high risk loan.

However, this results can be misleading bacause our dataset is imbalanced, meaning there are significantly more healthy loans than high-risk loans. Therefore, it's important to look at resampled dataset.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [37]:
y_train

,loan_status
44997,0
13468,0
52163,0
5330,0
9982,0
...,...
28028,0
18051,0
23757,0
6607,0


In [38]:
X_train

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
44997,8500.0,6.722,43900,0.316629,3,0,13900
13468,10000.0,7.360,49900,0.398798,4,0,19900
52163,8400.0,6.708,43700,0.313501,3,0,13700
5330,11000.0,7.779,53800,0.442379,5,1,23800
9982,8900.0,6.924,45800,0.344978,3,0,15800
...,...,...,...,...,...,...,...
28028,10000.0,7.389,50100,0.401198,4,1,20100
18051,8700.0,6.809,44700,0.328859,3,0,14700
23757,8900.0,6.897,45500,0.340659,3,0,15500
6607,10300.0,7.494,51100,0.412916,4,1,21100


In [39]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
#Counter(y_resampled)


In [40]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
credit_ROS = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_over, y_over = credit_ROS.fit_resample(X_train, y_train)

In [41]:
credit_ROS

RandomOverSampler(random_state=1)

In [43]:
# Count the distinct values of the resampled labels data
y_over["loan_status"].value_counts()

1    56277
0    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [44]:
# Instantiate the Logistic Regression model
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_over, y_over["loan_status"])

# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test["loan_status"]}).tail(10)

,Prediction,Actual
76038,1,1
19119,0,0
45193,0,0
25211,0,0
61697,0,0
38069,0,0
36892,0,0
5035,0,0
40821,0,0
35030,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model 
# YOUR CODE HERE!

In [ ]:
# Generate a confusion matrix for the model
# YOUR CODE HERE!

In [ ]:
# Print the classification report for the model
# YOUR CODE HERE!

### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** YOUR ANSWER HERE!